In [ ]:
import numpy as np 
import pandas as pd
import pickle
import uproot
import utilsv2
import utilsv5
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from matplotlib import gridspec

In [ ]:
# import argparse
# from data_prep import data_prep
# from utils_hh4bdata import get_category_presets
# import numpy as np
# import json

In [ ]:
def get_mu(file, year: int = 18, vr: bool = False, sample='med') -> float:
    """get nominal norm value from NNT"""
    vr_fix = "_VRderiv" if vr else ""
    if sample == 'med':
        return file[f"NN_norm{vr_fix}_bstrap_{sample}_{year}"].member("fVal")
    else:
        return file[f"NN_norm{vr_fix}_resampling_{sample}_{year}"].member("fVal")

def add_100NN_weights(df, f_data, year):
    for i in range(100):
        norm = get_mu(f_data,year,sample=str(i))
        df[f'NN_weights_{i}'] = norm * df[f'NN_d24_weight_resampling_{i}_{year}']
    
    return df


def add_100NN_weights_VR(df, f_data, year):
    for i in range(100):
        norm = get_mu(f_data,year,sample=str(i),vr=True)
        df[f'NN_weights_{i}'] = norm * df[f'NN_d24_weight_VRderiv_resampling_{i}_{year}']
        
    return df
    



def get_data_mask_100NN(data18,mask='2bRW', vr=False):
    '''apply mask to the data
    also add sample_weight and class columns
    note: this function could be appended to get_data(), but we may want to have 
    different masks for the same files and not load the files everytime,
    e.g. 2bRW and 4b masks in control region
    inputs:
            data16,17,18: outputs of the get_data() function
            mask: 2bRW or 4b, default=2bRW
    outputs:
            df: dataset after specific masks and concatenate all three years data
    '''
    # concatenate data
    data_all = data18
    if mask=='2bRW':
        if vr==True:
            df = data_all.loc[(data_all["ntag"] == 2) & (data_all["rw_to_4b_VR"] == True)].reset_index(drop=True) 
        else:
            df = data_all.loc[(data_all["ntag"] == 2) & (data_all["rw_to_4b"] == True)].reset_index(drop=True)
        df['sample_weight'] = df['NN_weights']
        df['class'] = 0
#         for i in range(100):
#             df[f'sample_weight_{i}'] = df[f'NN_weights_{i}']
    if mask=='4b':
        df = data_all.loc[data_all['ntag']>=4].reset_index(drop=True)
        # signal weights and class
        df['sample_weight'] = 1
        df['class'] = 1
        
    return df

In [ ]:
bins = np.array([0, 0.0396, 0.0796, 0.1152, 0.1469, 0.175, 0.2, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.375, 0.4, 0.425, 0.45, 0.475, 0.5, 0.525, 0.55, 0.575, 0.6, 0.625, 0.651, 0.6782, 0.7064, 0.7359, 0.7665, 0.7985, 0.8317, 0.8664, 0.9025, 0.9401, 1])

In [ ]:
def poisson_err(df, weights, bins=bins):
    return np.sqrt(np.histogram(df, weights=weights ** 2, bins=bins)[0])

def shape_syst(h_CR, h_VR):
    return np.abs(h_CR-h_VR)

def get_tot_error(poisson_err, shape_syst):
    return np.sqrt(poisson_err**2 + shape_syst**2)

## s9 + dRjj_1 + dRjj_2

In [ ]:
#data files
# file_path_data_16 = "data16_NN_100_bootstraps.root"
# file_path_data_17 = "data17_NN_100_bootstraps.root"
file_path_data_18 = "data18_NN_100_bootstraps.root"

In [ ]:
# f_data16 = uproot.open("data16_NN_100_bootstraps.root")
# f_data17 = uproot.open("data17_NN_100_bootstraps.root")
f_data18 = uproot.open("data18_NN_100_bootstraps.root")
# file_path_data_16 = []
# file_path_data_17 = []
# file_path_data_18 = []
# for region in ["control", "validation", "sig"]:
#     with uproot.open("data16_NN_100_bootstraps.root:{}".format(region)) as file1:
#         df = pd.DataFrame(file1.arrays(file1.keys(), "ntag != 3", library='np'))
#         file_path_data_16.append(df)
    
#     with uproot.open("data17_NN_100_bootstraps.root:{}".format(region)) as file2:
#         df = pd.DataFrame(file2.arrays(file2.keys(), "ntag != 3", library='np'))
#         file_path_data_17.append(df)
    
#     with uproot.open("data18_NN_100_bootstraps.root:{}".format(region)) as file3:
#         df = pd.DataFrame(file3.arrays(file3.keys(), "ntag != 3", library='np'))
#         file_path_data_18.append(df)
        
# file_path_data_16 = pd.concat(file_path_data_16, axis=0)
# file_path_data_17 = pd.concat(file_path_data_17, axis=0)
# file_path_data_18 = pd.concat(file_path_data_18, axis=0)

In [ ]:
data18_CR=utilsv2.get_data(region='control',half='odd')

In [ ]:
#print(list(data18_CR.keys()))

In [ ]:
# data16_CR = add_100NN_weights(data16_CR,f_data16,16)
# data17_CR = add_100NN_weights(data17_CR,f_data17,17)
data18_CR = add_100NN_weights(data18_CR,f_data18,18)

In [ ]:
# change to your own path
model = tf.keras.models.load_model("./classifier_models/s9+dRjj_12_model")
scaler = pickle.load(open("./classifier_models/StandardScaler_s9+dRjj_12.pkl", "rb"))

In [ ]:
features = ['m_hh','X_hh','dEta_hh','X_wt_tag','year_16','year_17','year_18','bkt_0','bkt_1','pt_hh','m_h1','m_h2','dRjj_1',
            'dRjj_2'
#             'bkt_lead_jet_pt','bkt_third_lead_jet_pt','pT_h1',
#             'cos_theta_star','njets',
#            'pt_hh','pT_2','pT_4','eta_i','dRjj_1','dRjj_2','m_min_dj','m_max_dj',
#           'pairing_score_1','pairing_score_2',
#           'm_h1','E_h1','eta_h1','phi_h1','m_h2','E_h2','pT_h2','eta_h2','phi_h2',
#           'm_h1_j1','E_h1_j1','eta_h1_j1','phi_h1_j1',
#            'm_h1_j2','E_h1_j2','eta_h1_j2','phi_h1_j2',
#           'm_h2_j1','E_h2_j1','eta_h2_j1','phi_h2_j1',
#            'm_h2_j2','E_h2_j2','eta_h2_j2','phi_h2_j2','year'
           ]

## Prediction in CR1

In [ ]:
bkg2b_CR = get_data_mask_100NN(data18_CR,mask='2bRW')
#print(bkg2b_CR.keys().tolist())
#print(len(weights_2b_CR))
bkg4b_CR = get_data_mask_100NN(data18_CR,mask='4b')
df_data_CR = pd.concat([bkg2b_CR, bkg4b_CR], ignore_index=True)

In [ ]:
weights_2b_CR = bkg2b_CR['sample_weight']
weights_100NN_CR = {}
# for i in range(100):
#     weights_100NN_CR[f'weights_{i}'] = bkg2b_CR[f'sample_weight_{i}']

In [ ]:
X_CR = df_data_CR[features]
y_CR = df_data_CR['class']

In [ ]:
predict_CR = model.predict(scaler.fit_transform(X_CR))

predict_4b_CR = predict_CR[:,1][y_CR == 1]
predict_2b_CR = predict_CR[:,1][y_CR == 0]

In [ ]:
# %store predict_4b_CR
# %store predict_2b_CR

In [ ]:
fig = plt.figure(figsize = (10,9))
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
ax = [0,0]
ax[0] = plt.subplot(gs[0])
ax[1] = plt.subplot(gs[1])

# bins = np.linspace(0.0,1.0,126)
e = bins

# for i in range(100):
#     ax[0].hist(predict_2b_CR, bins = bins, histtype='step',weights = weights_100NN_CR[f'weights_{i}'], color ='orange',ls='--')

h1_CR, be = np.histogram(predict_4b_CR, bins = bins)
h2_CR,_,_ = ax[0].hist(predict_2b_CR, bins = be, histtype='stepfilled',weights = weights_2b_CR, label = '2bRW data', fc='gold',ec='k')
#n_sig, bins_SR, _ = ax[0].hist(mc_CR, histtype='stepfilled',weights=mc_CR.weight, bins= bins, fc='gold',ec='k')
scaling_CR = h1_CR/h2_CR

scaling_VR = [0.79672866, 0.96214765, 0.96014945, 1.02487936, 0.99818721, 1.05582024,
 1.02111186, 0.95481258, 0.95930657, 1.00102865, 1.05316608, 1.0143762,
 0.99979671, 0.97035153, 0.99057374, 1.00988367, 1.03159916, 0.96051671,
 1.04903227, 1.03692782, 1.0732052,  1.0528787,  1.04286564, 1.01930452,
 0.96425807, 1.08786646, 1.08239959, 1.04547539, 1.10899099, 1.0299609,
 1.03728416, 1.00490792, 1.11551188, 1.24163119]

h_CR_VRrw,_ = np.histogram(bins[:-1],bins=bins,weights=h2_CR*scaling_VR)
poisson_CR = poisson_err(predict_2b_CR, weights_2b_CR, bins=bins)
shape_CR = shape_syst(h2_CR, h_CR_VRrw)
# bstrap_VR = np.array(sbs_VR['classifier_prob']['all']['inclusive']) * h2_VR
tot_err_CR = get_tot_error(poisson_CR, shape_CR)


ax[0].fill_between(
                    bins,
                    np.append(h2_CR - tot_err_CR, 0),
                    np.append(h2_CR + tot_err_CR, 0),
                    hatch="\\\\\\\\",
                    facecolor="None",
                    edgecolor="dimgrey",
                    linewidth=0,
                    step="post",
                    zorder=1,
                    label = "Stat + Shape Error"
                )
ax[0].errorbar((bins[1:] + bins[:-1])/2,h1_CR,yerr=np.sqrt(h1_CR),xerr=(bins[1:] - bins[:-1]) / 2,fmt="o",ms=4,c='k',label='4b data')


# _ = ax[0].hist(bins[:-1],bins=bins,weights=h2_CR*scaling_VR,histtype='step', label = 'VR RW', ls = '--',color='purple')

ax[1].axhline(1, linestyle='dashed', color='black')
ax[1].set_ylim(0.5,1.5)

ax[1].errorbar((bins[1:] + bins[:-1])/2,
                scaling_CR,
                yerr = np.sqrt(h1_CR)/h1_CR,
                xerr=(bins[1:] - bins[:-1]) / 2,
                fmt="o",
                ms=4,
                c='k',
                label='4b/2bRW'
            )
ax[1].fill_between(
                bins,
                np.append((h2_CR - tot_err_CR) / h2_CR, 0),
                np.append((h2_CR + tot_err_CR) / h2_CR, 0),
                hatch="\\\\\\\\",
                facecolor="None",
                edgecolor="dimgrey",
                linewidth=0,
                step="post",
                zorder=1,
            )


ax[0].legend()
ax[1].legend(loc="upper left")
ax[1].set_xlabel('NN predicted probability', fontsize=18)
ax[1].set_ylabel('ratio', fontsize=18)
ax[0].set_ylabel('Frequency', fontsize = 18)
ax[0].set_title('CR1 Distribution Comparison ', fontsize=18)


plt.savefig("Classifier_CR1_hist.png", dpi = 300)

## Prediction in CR2

In [ ]:
data18_VR=utilsv2.get_data(region='validation',vr=True)

In [ ]:
# data16_VR = add_100NN_weights(data16_VR,f_data16,16)
# data17_VR = add_100NN_weights(data17_VR,f_data17,17)
data18_VR = add_100NN_weights_VR(data18_VR,f_data18,18)

In [ ]:
# %store data16_VR
# %store data17_VR
# with open('file', 'wb') as outfile:
#     pickle.dump(data18_VR, outfile, protocol=4)

In [ ]:
# %store -r data16_VR
# %store -r data17_VR
# with open('file', 'rb') as file:
#     data18_VR = pickle.load(file)

In [ ]:
bkg2b_VR = get_data_mask_100NN(data18_VR,mask='2bRW', vr=True)
weights_2b_VR = bkg2b_VR['sample_weight']
bkg4b_VR = get_data_mask_100NN(data18_VR,mask='4b')
df_data_VR = pd.concat([bkg2b_VR, bkg4b_VR], ignore_index=True)

In [ ]:
weights_2b_VR = bkg2b_VR['sample_weight']
weights_100NN_VR = {}
# for i in range(100):
#     weights_100NN_VR[f'weights_{i}'] = bkg2b_VR[f'sample_weight_{i}']

In [ ]:
X_VR = df_data_VR[features]
y_VR = df_data_VR['class']

In [ ]:
predict_VR = model.predict(scaler.fit_transform(X_VR))

predict_4b_VR = predict_VR[:,1][y_VR == 1]
predict_2b_VR = predict_VR[:,1][y_VR == 0]

In [ ]:
# %store -r predict_4b_CR
# %store -r predict_2b_CR

In [ ]:
fig = plt.figure(figsize = (10,9))
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
ax = [0,0]
ax[0] = plt.subplot(gs[0])
ax[1] = plt.subplot(gs[1])


e = bins
 

h1_VR, be = np.histogram(predict_4b_VR, bins = bins)
h2_VR,_,_ = ax[0].hist(predict_2b_VR, bins = be, histtype='stepfilled',weights = weights_2b_VR, label = '2bRW data', fc='gold',ec='k')
scaling_VR = h1_VR/h2_VR

scaling_CR = [0.75680029, 1.03215854, 1.00161104, 1.03996201, 0.99909953, 0.9702849,
 0.99605103, 1.03652869, 1.05054138, 1.01813334, 1.01085846, 0.96804955,
 1.01648839, 1.00061676, 1.05924889, 0.95336036, 1.00089076, 1.01191376,
 1.03812849, 0.91751007, 0.98630383, 1.06191841, 1.09371214, 0.95501103,
 0.99413626, 0.99047827, 1.00030871, 1.01609331, 1.06519411, 0.97342179,
 1.04430647, 1.02516602, 1.33708051, 1.73349743]

h_VR_CRrw,_ = np.histogram(bins[:-1],bins=bins,weights=h2_VR*scaling_CR)

poisson_VR = poisson_err(predict_2b_VR, weights_2b_VR, bins=bins)
shape_VR = shape_syst(h2_VR, h_VR_CRrw)
# bstrap_VR = np.array(sbs_VR['classifier_prob']['all']['inclusive']) * h2_VR
tot_err_VR = get_tot_error(poisson_VR, shape_VR)

ax[0].fill_between(
                    bins,
                    np.append(h2_VR - tot_err_VR, 0),
                    np.append(h2_VR + tot_err_VR, 0),
                    hatch="\\\\\\\\",
                    facecolor="None",
                    edgecolor="dimgrey",
                    linewidth=0,
                    step="post",
                    zorder=1,
                    label = "Stat + Shape Error"
                )
ax[0].errorbar((bins[1:] + bins[:-1])/2,h1_VR,yerr=np.sqrt(h1_VR),xerr=(bins[1:] - bins[:-1]) / 2,fmt="o",ms=4,c='k',label='4b data')

 


 

ax[1].axhline(1, linestyle='dashed', color='black')
ax[1].set_ylim(0.5,1.5)

 

ax[1].errorbar((bins[1:] + bins[:-1])/2,
                scaling_VR,
                yerr = np.sqrt(h1_VR)/h1_VR,
                xerr=(bins[1:] - bins[:-1]) / 2,
                fmt="o",
                ms=4,
                c='k',
                label='4b/2bRW'
            )
ax[1].fill_between(
                bins,
                np.append((h2_VR - tot_err_VR) / h2_VR, 0),
                np.append((h2_VR + tot_err_VR) / h2_VR, 0),
                hatch="\\\\\\\\",
                facecolor="None",
                edgecolor="dimgrey",
                linewidth=0,
                step="post",
                zorder=1,
            )

 


ax[0].legend()
ax[1].legend(loc="upper left")
ax[1].set_xlabel('NN predicted probability', fontsize=18)
ax[1].set_ylabel('ratio', fontsize=18)
ax[0].set_ylabel('Frequency', fontsize = 18)
ax[0].set_title('CR2 Distribution Comparison', fontsize=18)

 
plt.savefig("Classifier_CR2_hist.png", dpi = 300)

## Prediction in merged CR

In [ ]:
predict_4b_tot = np.concatenate([predict_4b_CR, predict_4b_VR])
predict_2b_tot = np.concatenate([predict_2b_CR, predict_2b_VR])
weights_2b_tot = np.concatenate([weights_2b_CR, weights_2b_VR])

In [ ]:
fig = plt.figure(figsize = (10,9))
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
ax = [0,0]
ax[0] = plt.subplot(gs[0])
ax[1] = plt.subplot(gs[1])


e = bins
 

h1_tot, be = np.histogram(predict_4b_tot, bins = bins)
h2_tot,_,_ = ax[0].hist(predict_2b_tot, bins = be, histtype='stepfilled',weights = weights_2b_tot, label = '2bRW data', fc='gold',ec='k')
scaling_tot = h1_tot/h2_tot


#h_VR_CRrw,_ = np.histogram(bins[:-1],bins=bins,weights=h2_VR*scaling_CR)

poisson_tot = poisson_err(predict_2b_tot, weights_2b_tot, bins=bins)
#shape_VR = shape_syst(h2_VR, h_VR_CRrw)
# bstrap_VR = np.array(sbs_VR['classifier_prob']['all']['inclusive']) * h2_VR
# tot_err_VR = get_tot_error(poisson_VR, shape_VR, bstrap_VR)

ax[0].fill_between(
                    bins,
                    np.append(h2_tot - poisson_tot, 0),
                    np.append(h2_tot + poisson_tot, 0),
                    hatch="\\\\\\\\",
                    facecolor="None",
                    edgecolor="dimgrey",
                    linewidth=0,
                    step="post",
                    zorder=1,
                    label = "Stat Error"
                )
ax[0].errorbar((bins[1:] + bins[:-1])/2,h1_tot,yerr=np.sqrt(h1_tot),xerr=(bins[1:] - bins[:-1]) / 2,fmt="o",ms=4,c='k',label='4b data')

 


 

ax[1].axhline(1, linestyle='dashed', color='black')
ax[1].set_ylim(0.5,1.5)

 

ax[1].errorbar((bins[1:] + bins[:-1])/2,
                scaling_tot,
                yerr = np.sqrt(h1_tot)/h1_tot,
                xerr=(bins[1:] - bins[:-1]) / 2,
                fmt="o",
                ms=4,
                c='k',
                label='4b/2bRW'
            )
ax[1].fill_between(
                bins,
                np.append((h2_tot - poisson_tot) / h2_tot, 0),
                np.append((h2_tot + poisson_tot) / h2_tot, 0),
                hatch="\\\\\\\\",
                facecolor="None",
                edgecolor="dimgrey",
                linewidth=0,
                step="post",
                zorder=1,
            )

 


ax[0].legend()
ax[1].legend(loc="upper left")
ax[1].set_xlabel('NN predicted probability', fontsize=18)
ax[1].set_ylabel('ratio', fontsize=18)
ax[0].set_ylabel('Frequency', fontsize = 18)
ax[0].set_title('Merged CR Distribution Comparison', fontsize=18)

 
plt.savefig("Classifier_CRtot_hist.png", dpi = 300)

## Prediction in SR

### with CR1 weights

In [ ]:
data18_SR=utilsv2.get_data(region='sig')

In [ ]:
bkg2b_SR_CR = get_data_mask_100NN(data18_SR,mask='2bRW')
weights_2b_SR_CR = bkg2b_SR_CR['sample_weight']
bkg4b_SR_CR = get_data_mask_100NN(data18_SR,mask='4b')
# df_data_SR_CR = pd.concat([bkg2b_SR_CR, bkg4b_SR_CR], ignore_index=True)

# bkg2b_SR_VR = get_data_mask_100NN(data18_SR,mask='2bRW', vr=True)
# weights_2b_SR_VR = bkg2b_SR_VR['sample_weight']
# bkg4b_SR_VR = get_data_mask_100NN(data18_SR_VR,mask='4b')
# df_data_SR_VR = pd.concat([bkg2b_SR_VR, bkg4b_SR_VR], ignore_index=True)

In [ ]:
mc_test=utilsv5.get_data("nominal_3.root",region='sig',mc=True)

In [ ]:
#bkg2b_SR_mc = utilsv4.get_data_mask(mc_SR,mask='2bRW')
#weights_2b_SR_mc = bkg2b_SR_mc['sample_weight']

mc_df = utilsv5.get_data_mask(mc_test,mask='4b')
#weights_mc = mc_df.weight
#df_data_SR_mc = bkg4b_SR_mc
mc_df["class"]=2
# bkg2b_SR_VR = get_data_mask_100NN(data18_SR,mask='2bRW', vr=True)
# weights_2b_SR_VR = bkg2b_SR_VR['sample_weight']
# bkg4b_SR_VR = get_data_mask_100NN(data18_SR_VR,mask='4b')
# df_data_SR_VR = pd.concat([bkg2b_SR_VR, bkg4b_SR_VR], ignore_index=True)

In [ ]:
df_data_SR = pd.concat([bkg2b_SR_CR, bkg4b_SR_CR, mc_df], ignore_index=True)

In [ ]:
X_SR = df_data_SR[features]
y_SR = df_data_SR['class']

In [ ]:
predict_SR = model.predict(scaler.fit_transform(X_SR))

predict_4b_SR = predict_SR[:,1][y_SR == 1]
predict_2b_SR = predict_SR[:,1][y_SR == 0]
predict_mc = predict_SR[:,1][y_SR == 2]

In [ ]:
#print(mc_df.weight)

In [ ]:
#print(weights_mc)
#print(predict)
#weights_mc = weights_mc.apply(np.sum)
#print(np.sum(weights_mc[0]))

In [ ]:
fig = plt.figure(figsize = (10,9))
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
ax = [0,0]
ax[0] = plt.subplot(gs[0])
ax[1] = plt.subplot(gs[1])


e = bins
 

h1_SR, be = np.histogram(predict_4b_SR, bins = bins)
h2_SR,_,_ = ax[0].hist(predict_2b_SR, bins = be, alpha = 0.5, histtype='stepfilled', weights = weights_2b_SR_CR, label = '2bRW data', fc='gold',ec='k')
scaling_SR = h1_SR/h2_SR
hmc, _, _ = ax[0].hist(predict_mc, bins=be, alpha = 0.5, histtype='stepfilled', weights = mc_df.weight*100, label="MC 4b data (scaled by 100)", fc = "red", ec="k")

scaling_SR_VR = [0.78876978, 1.01373882, 1.05521853, 1.0525372, 1.08652082, 1.20017687,
         1.04220829, 1.17883227, 1.00138928, 0.97655062, 1.12259732, 1.06373665,
         1.0421137,  1.07359879, 1.1859636, 1.04714045, 0.90925962, 1.15842675,
         1.07726025, 1.12011166, 1.05216968, 1.1986841,  1.13148614, 0.869011,
         1.10913564, 1.03434718, 1.16903833, 1.27699274, 1.16100603, 1.42171882,
         1.42086023, 1.36821049, 1.55373942, 1.32026782]


h_SR_VRrw,_ = np.histogram(bins[:-1],bins=bins,weights=h2_SR*scaling_SR_VR)

poisson_SR = poisson_err(predict_2b_SR, weights_2b_SR_CR, bins=bins)
shape_SR = shape_syst(h2_SR, h_SR_VRrw)
# bstrap_VR = np.array(sbs_VR['classifier_prob']['all']['inclusive']) * h2_VR
tot_err_SR = get_tot_error(poisson_SR, shape_SR)
            

ax[0].fill_between(
                    bins,
                    np.append(h2_SR - tot_err_SR, 0),
                    np.append(h2_SR + tot_err_SR, 0),
                    hatch="\\\\\\\\",
                    facecolor="None",
                    edgecolor="dimgrey",
                    linewidth=0,
                    step="post",
                    zorder=1,
                    label = "Stat + Shape Error"
                )

ax[0].errorbar((bins[1:] + bins[:-1])/2,h1_SR,yerr=np.sqrt(h1_SR),xerr=(bins[1:] - bins[:-1]) / 2,fmt="o",ms=4,c='k',label='4b data')

 


 

ax[1].axhline(1, linestyle='dashed', color='black')
ax[1].set_ylim(0.5,1.5)

 

ax[1].errorbar((bins[1:] + bins[:-1])/2,
                scaling_SR,
                yerr = np.sqrt(h1_SR)/h1_SR,
                xerr=(bins[1:] - bins[:-1]) / 2,
                fmt="o",
                ms=4,
                c='k',
                label='4b/2bRW'
            )
ax[1].fill_between(
                bins,
                np.append((h2_SR - tot_err_SR) / h2_SR, 0),
                np.append((h2_SR + tot_err_SR) / h2_SR, 0),
                hatch="\\\\\\\\",
                facecolor="None",
                edgecolor="dimgrey",
                linewidth=0,
                step="post",
                zorder=1,
            )

 


ax[0].legend()
ax[1].legend(loc="upper left")
ax[1].set_xlabel('NN predicted probability', fontsize=18)
ax[1].set_ylabel('ratio', fontsize=18)
ax[0].set_ylabel('Frequency', fontsize = 18)
ax[0].set_title('SR Distribution Comparison with CR1 weights', fontsize=18)

 
plt.savefig("Classifier_SR_CR1_hist.png", dpi = 300)

In [ ]:
sensitivity = 0
for i in range (0,len(bins)-1):
    sensitivity += hmc[i]/np.sqrt(h2_SR[i])
    
print("Sensitivity of this classifier model is",sensitivity/100)

### with CR2 weights

In [ ]:
data18_SR_VR=utilsv2.get_data(region='sig',vr=True)

In [ ]:
bkg2b_SR_VR = get_data_mask_100NN(data18_SR_VR,mask='2bRW', vr=True)
weights_2b_SR_VR = bkg2b_SR_VR['sample_weight']
bkg4b_SR_VR = get_data_mask_100NN(data18_SR_VR,mask='4b')

In [ ]:
df_data_SR_VR = pd.concat([bkg2b_SR_VR, bkg4b_SR_VR, mc_df], ignore_index=True)

In [ ]:
X_SR_VR = df_data_SR_VR[features]
y_SR_VR = df_data_SR_VR['class']

In [ ]:
predict_SR_VR = model.predict(scaler.fit_transform(X_SR_VR))

predict_4b_SR_VR = predict_SR_VR[:,1][y_SR_VR == 1]
predict_2b_SR_VR = predict_SR_VR[:,1][y_SR_VR == 0]
predict_mc = predict_SR_VR[:,1][y_SR_VR == 2]

In [ ]:
fig = plt.figure(figsize = (10,9))
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
ax = [0,0]
ax[0] = plt.subplot(gs[0])
ax[1] = plt.subplot(gs[1])


e = bins
 

h1_SR_VR, be = np.histogram(predict_4b_SR_VR, bins = bins)
h2_SR_VR,_,_ = ax[0].hist(predict_2b_SR_VR, bins = be, alpha = 0.5, histtype='stepfilled', weights = weights_2b_SR_VR, label = '2b RW data', fc='gold',ec='k')
scaling_SR_VR = h1_SR_VR/h2_SR_VR
hmc, _, _ = ax[0].hist(predict_mc, bins=be, alpha = 0.5, histtype='stepfilled', weights = mc_df.weight*100, label="MC 4b data", fc = "red", ec="k")

scaling_SR_CR = [0.73806214, 0.97987849, 1.00689351, 1.0012123, 1.03888799, 1.14994957,
         0.99279614, 1.11915548, 0.95214903, 0.93115445, 1.06934413, 1.01972296,
         1.00151906, 1.02974421, 1.13860969, 0.99819294, 0.87957235, 1.12296621,
         1.04404993, 1.08754922, 1.03267916, 1.18021076, 1.11690424, 0.85774974,
         1.09547353, 1.02628903, 1.16057627, 1.26905884, 1.16164742, 1.41583747,
         1.41185113, 1.36336849, 1.55356767, 1.32327288]

h_SR_CRrw,_ = np.histogram(bins[:-1],bins=bins,weights=h2_SR_VR*scaling_SR_CR)

poisson_SR_VR = poisson_err(predict_2b_SR_VR, weights_2b_SR_VR, bins=bins)
shape_SR_VR = shape_syst(h2_SR_VR, h_SR_CRrw)
# bstrap_VR = np.array(sbs_VR['classifier_prob']['all']['inclusive']) * h2_VR
tot_err_SR_VR = get_tot_error(poisson_SR_VR, shape_SR_VR)
            

ax[0].fill_between(
                    bins,
                    np.append(h2_SR_VR - poisson_SR_VR, 0),
                    np.append(h2_SR_VR + poisson_SR_VR, 0),
                    hatch="\\\\\\\\",
                    facecolor="None",
                    edgecolor="dimgrey",
                    linewidth=0,
                    step="post",
                    zorder=1,
                    label = "Stat + Shape Error"
                )

ax[0].errorbar((bins[1:] + bins[:-1])/2,h1_SR_VR,yerr=np.sqrt(h1_SR_VR),xerr=(bins[1:] - bins[:-1]) / 2,fmt="o",ms=4,c='k',label='4b data')


 


 

ax[1].axhline(1, linestyle='dashed', color='black')
ax[1].set_ylim(0.5,1.5)

 

ax[1].errorbar((bins[1:] + bins[:-1])/2,
                scaling_SR_VR,
                yerr = np.sqrt(h1_SR_VR)/h1_SR_VR,
                xerr=(bins[1:] - bins[:-1]) / 2,
                fmt="o",
                ms=4,
                c='k',
                label='4b/2bRW'
            )
ax[1].fill_between(
                bins,
                np.append((h2_SR_VR - tot_err_SR_VR) / h2_SR_VR, 0),
                np.append((h2_SR_VR + tot_err_SR_VR) / h2_SR_VR, 0),
                hatch="\\\\\\\\",
                facecolor="None",
                edgecolor="dimgrey",
                linewidth=0,
                step="post",
                zorder=1,
            )

 


ax[0].legend()
ax[1].legend(loc="upper left")
ax[1].set_xlabel('NN predicted probability', fontsize=18)
ax[1].set_ylabel('ratio', fontsize=18)
ax[0].set_ylabel('Frequency', fontsize = 18)
ax[0].set_title('SR Distribution Comparison with CR2 weights', fontsize=18)

 
plt.savefig("Classifier_SR_CR2_hist.png", dpi = 300)

In [ ]:
sensitivity = 0
for i in range (0,len(bins)-1):
    sensitivity += hmc[i]/np.sqrt(h2_SR_VR[i])
    
print("Sensitivity of this classifier model is",sensitivity/100)

## Sensitivty calculation

In [ ]:
tree_name='sig'
# list_of_columns_to_load = ['m_hh', 'ntag']
preCut="( ntag >= 4 )&(X_wt_tag>1.5)&(pass_vbf_sel==False)" 
with uproot.open("nominal_3.root") as root_file:
    df = root_file[tree_name].arrays(cut=preCut, library='pd')
    mc_sf_np = np.atleast_2d(
                np.stack(
                    (root_file[tree_name].arrays(["mc_sf"], cut=preCut, library="np"))[
                        "mc_sf"                    ]
                ).T            )
    nominal_mc_sf = mc_sf_np[0]
    df['weight'] = 57.70 * nominal_mc_sf
    
#print(df)


In [ ]:
tree_name='sig'
list_of_columns_to_load = ['m_hh', 'ntag','NN_d24_weight_bstrap_med_18',"pass_vbf_sel","rw_to_4b"]
preCut="(ntag ==2)&(X_wt_tag>1.5)&(rw_to_4b==True)&(pass_vbf_sel == False)" 
with uproot.open("data18_NN_100_bootstraps.root") as root_file:
    df2b = root_file[tree_name].arrays(list_of_columns_to_load, cut=preCut, library='pd')
    rw_norm = root_file['NN_norm_bstrap_med_18'].value
    df2b['weight'] = df2b['NN_d24_weight_bstrap_med_18']*rw_norm


In [ ]:
bins_SR = [280.0, 305.0, 333.0, 363.0, 395.0, 431.0, 470.0, 512.0, 558.0, 608.0, 663.0, 723.0, 788.0, 858.0, 936.0, 1020.0]

In [ ]:
fig = plt.figure(figsize = (10,7))
n_sig, bins_SR, _ = plt.hist(df.m_hh, alpha=0.7, histtype='stepfilled',weights=df.weight*100, bins= bins_SR, label="MC 4b data (signal scaled by 100)",fc='red',ec='k')
n_background, bins, _ = plt.hist(df2b.m_hh, alpha=0.5, histtype='stepfilled', weights=df2b.weight, bins= bins_SR, label="2bRW data (background)", fc='gold',ec='k')
plt.xlabel("m_hh (GeV)",fontsize=18)
plt.ylabel("Frequency", fontsize=18)
plt.legend(loc="best")
plt.title("Signal Region m_hh mass plane of signal and background data",fontsize=18)
plt.savefig("Standard_SR_hist.png", dpi = 300)

In [ ]:
sensitivity = 0
for i in range (0,len(bins)-1):
    sensitivity += n_sig[i]/np.sqrt(n_background[i])
    
print("Sensitivity of this classifier model is",sensitivity/100)